In [ ]:
import warnings
warnings.filterwarnings('ignore')
# final destination for local functions
import transect_viz
from transect_viz import transect_generator

#for comps
import numpy as np
import pandas as pd
import geopandas as gpd

# for viz
import holoviews as hv
from holoviews import opts, dim
import panel as pn
pn.extension(sizing_mode='stretch_width')
import hvplot.pandas

# for caching
import functools
import os

In [ ]:
def read_geojson(file):
    '''
    read geojson file and return it in crs of UTM Zone 10N, i.e. epsg=32610'''
    gdf = gpd.read_file(file)
    return gdf.to_crs('epsg:32610')

In [ ]:
def create_points_along_line(line, delx=25):
    '''
    line is a shapely line string
    create equidistant points (delx (25) apart) along line by projection 
    return a geodata frame in crs of epsg 32610 (UTM Zone 10N)
    '''
    return gpd.GeoDataFrame(data={'transect_dist':np.arange(0,line.length,delx)},
                            geometry=[line.interpolate(x) for x in np.arange(0,line.length,delx)], crs='epsg:32610')

In [ ]:
line = read_geojson('../geodata/sugarcut.geojson')

In [ ]:
#line.hvplot(geo=True, tiles=True)

In [ ]:
#convert to UTM Zone 10N to get length in feet

In [ ]:
print(line.length[0])

In [ ]:
#line.hvplot(crs='+init=epsg:32610', geo=True, tiles=True)

In [ ]:
gdfp = create_points_along_line(line.iloc[0].geometry)
gdfp.head(2)

In [ ]:
gdfp.info()

In [ ]:
ptsmap = hv.element.tiles.CartoLight()*gdfp.hvplot(crs='+init=epsg:32610',geo=True)*line.hvplot(crs='+init=epsg:32610',geo=True)

In [ ]:
ptsmap

In [ ]:
#add longitude and latitude for geoviews to work
def add_lon_lat(gdfp):
    gdfp_ll = gdfp.copy().to_crs('+init=epsg:4326')
    gdfp['Longitude']=gdfp_ll.geometry.x
    gdfp['Latitude']=gdfp_ll.geometry.y
    return gdfp

In [ ]:
gdfp = add_lon_lat(gdfp)

In [ ]:
# saving generated points (equally spaced) to geojson file
gdfp.to_file("../geodata/sugarcut_pts.geojson", driver='GeoJSON')

In [ ]:
gdfp2 = gpd.read_file('../geodata/sugarcut_pts.geojson')
gdfp2.head(3)

In [ ]:
from transect_viz import transect_cdec_data
def load_stations(stations):
    dfs = transect_cdec_data.get_stations(stations)
    gdfs = transect_generator.to_geoframe(dfs)
    #gdfs=gdfs.drop(columns=['Longitude','Latitude'])
    return gdfs

In [ ]:
station_ids = ['DAR', 'SUR', 'SGA']
gdfs = load_stations(station_ids)

In [ ]:
#ptsmap*
#gdfs.hvplot.points(crs="+init=epsg:32610",geo=True,tiles=True)
#print(gdfs.hvplot(geo=True))#,tiles=True)

#print(ptsmap)

In [ ]:
gdfs

In [ ]:
gdfp.crs

In [ ]:
#gdfp.hvplot(crs='+init=epsg:32610',geo=True)*gdfs.hvplot(crs='+init=epsg:32610',geo=True)
gdfp.hvplot(geo=True)*gdfs.hvplot(geo=True) # looks like uses latitude/longitude by default over the geometry column

In [ ]:
# create a combined data frame of all points with Station ID and values column
gdfs=transect_generator.add_transect_dist(gdfs, line.iloc[0].geometry)
gdfa = pd.concat([gdfs,gdfp]).set_index('transect_dist').sort_index()
gdfa.head(3)

In [ ]:
#@pn.io.cache # not available till panel 0.14.1+
#def get_station_data(sdate, edate, station_ids, data_type):
#    # get values for the Station ID and set values in this to those
#    dflist = transect_cdec_data.get_cdec_data_cached(sdate, edate, station_ids=station_ids, data_type=data_type)
#    dflist = [df.resample('15T').mean() for df in dflist]
#    df = pd.concat(dflist, axis=1)
#    return df

In [ ]:
# cache into pickled file
#def get_station_data_cached(sdate, edate, station_ids, data_type):
#    pfname = 'dataset_'+'_'.join(station_ids)+'_'.join([sdate,edate,data_type])+'.pkl'
#    if os.path.exists(pfname):
#        dfec = pd.read_pickle(pfname)
#    else:
#        dfec = get_station_data(sdate, edate, station_ids, data_type)
#        dfec.to_pickle(pfname)
#    return dfec

In [ ]:
sdate, edate = '01-01-2022','10-07-2022'
data_type='EC'
date_value = '2022-10-05 12:45:00'

In [ ]:
def interpolate_values(date_value, gdfa, dfdata):
    '''
    Uses interpolation over index on gdfa for the dfdata value at the date_value
    Adds a 'values' column to the gdfa data set 
    '''
    df = dfdata.loc[date_value].to_frame()
    df.columns=['values']
    gdfx = gdfa.join(df, on='Station ID')
    gdfx['values']=gdfx['values'].interpolate('index')
    gdfx['DateTime']=date_value
    return gdfx

In [ ]:
dfec = transect_cdec_data.get_cdec_data_cached(sdate, edate, station_ids, data_type)
gdfx = interpolate_values(date_value, gdfa, dfec)

In [ ]:
gdfx

In [ ]:
pts_map, pts_legend = transect_viz.map_transect_with_size_and_color(gdfx,data_column='values')

In [ ]:
hv.element.tiles.CartoLight()*pts_map+pts_legend